# ASL Real-Time Inference (250 Classes)

This notebook tests the pretrained Kaggle 1st-place ASL model with real-time webcam inference.

**Features:**
- MediaPipe landmark extraction
- Transformer model (250 ASL signs)
- 7 real-time inference tricks
- Webcam integration

**Run Order:** Execute cells 1-8 in sequence.

In [ ]:
# Cell 1: Install Dependencies
# Run this once, then restart runtime if needed

try:
    import tensorflow as tf
    import mediapipe as mp
    import cv2
    print("✅ All packages already installed!")
except (ImportError, ValueError):
    print("Installing dependencies...")
    !pip uninstall -y numpy
    !pip install numpy==1.26.4
    !pip install -q mediapipe opencv-python
    print("\n" + "="*50)
    print("⚠️ Click 'Runtime' → 'Restart runtime'")
    print("Then continue from Cell 2!")
    print("="*50)

In [ ]:
# Cell 2: Imports

import tensorflow as tf
import mediapipe as mp
import cv2
import numpy as np
import json
from collections import deque
import time
import os

print(f"✅ Imports successful!")
print(f"TensorFlow: {tf.__version__}")
print(f"NumPy: {np.__version__}")
print(f"GPU: {len(tf.config.list_physical_devices('GPU')) > 0}")

In [ ]:
# Cell 3: Setup Model Weights & Labels

import json
import os
from google.colab import files

# Model weights file (upload to /content/ folder)
WEIGHTS_FILE = "/content/islr-fp16-192-8-seed45-foldall-last.h5"

# Check if weights exist
if not os.path.exists(WEIGHTS_FILE):
    print("⚠️ Model weights not found!")
    print("📤 Please upload 'islr-fp16-192-8-seed45-foldall-last.h5' to Colab:")
    print("   1. Click the folder icon on the left sidebar")
    print("   2. Click the upload button")
    print("   3. Select the .h5 file from your computer")
    print("   4. Wait for upload to complete")
    print("   5. Re-run this cell")
    raise RuntimeError("Weights file not found. Please upload it first.")
else:
    file_size = os.path.getsize(WEIGHTS_FILE)
    print(f"✅ Model weights found: {file_size / (1024*1024):.1f} MB")

# Download competition labels (250 signs)
print("\n📥 Downloading 250 sign labels from Kaggle competition...")

# Setup Kaggle credentials if needed
if not os.path.exists('/root/.kaggle/kaggle.json'):
    print("📤 Upload your kaggle.json file:")
    uploaded = files.upload()
    if 'kaggle.json' in uploaded:
        !mkdir -p ~/.kaggle
        !mv kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        print("✅ Kaggle credentials configured!")
    else:
        raise RuntimeError("kaggle.json not uploaded")

# Install Kaggle API
!pip install -q kaggle

# Download the sign mapping file
LABEL_FILE = "sign_to_prediction_index_map.json"

if os.path.exists(LABEL_FILE):
    print(f"Found existing {LABEL_FILE}")
else:
    print(f"Downloading {LABEL_FILE} from asl-signs competition...")
    !kaggle competitions download -c asl-signs -f sign_to_prediction_index_map.json -p .
    !unzip -o sign_to_prediction_index_map.json.zip
    !rm sign_to_prediction_index_map.json.zip

# Load labels
with open(LABEL_FILE, 'r') as f:
    sign_to_index = json.load(f)

# Create reverse mapping (index -> sign)
index_to_sign = {str(v): k for k, v in sign_to_index.items()}

print(f"\n✅ Setup complete!")
print(f"   Model weights: {WEIGHTS_FILE}")
print(f"   Labels: {len(sign_to_index)} signs")
print(f"   Sample signs: {list(sign_to_index.keys())[:10]}")
print(f"\nYou can now use:")
print(f"   - sign_to_index: dict mapping sign names to indices (0-249)")
print(f"   - index_to_sign: dict mapping indices to sign names")

In [ ]:
# Cell 4: hoyso48 EXACT Model Architecture (from his Kaggle notebook)

# Constants
ROWS_PER_FRAME = 236  # 708 / 3 = 236 landmarks
MAX_LEN = 384
CHANNELS = ROWS_PER_FRAME * 3  # 708
NUM_CLASSES = 250
PAD = 0  # Padding value for masking


# ==========================================
# CUSTOM LAYERS (exact copy from hoyso48)
# ==========================================

class ECA(tf.keras.layers.Layer):
    def __init__(self, kernel_size=5, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.kernel_size = kernel_size
        self.conv = tf.keras.layers.Conv1D(1, kernel_size=kernel_size, strides=1, padding="same", use_bias=False)

    def call(self, inputs, mask=None):
        nn = tf.keras.layers.GlobalAveragePooling1D()(inputs, mask=mask)
        nn = tf.expand_dims(nn, -1)
        nn = self.conv(nn)
        nn = tf.squeeze(nn, -1)
        nn = tf.nn.sigmoid(nn)
        nn = nn[:,None,:]
        return inputs * nn


class LateDropout(tf.keras.layers.Layer):
    def __init__(self, rate, noise_shape=None, start_step=0, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.rate = rate
        self.start_step = start_step
        self.dropout = tf.keras.layers.Dropout(rate, noise_shape=noise_shape)
      
    def build(self, input_shape):
        super().build(input_shape)
        agg = tf.VariableAggregation.ONLY_FIRST_REPLICA
        self._train_counter = tf.Variable(0, dtype="int64", aggregation=agg, trainable=False)

    def call(self, inputs, training=False):
        x = tf.cond(self._train_counter < self.start_step, lambda:inputs, lambda:self.dropout(inputs, training=training))
        if training:
            self._train_counter.assign_add(1)
        return x


class CausalDWConv1D(tf.keras.layers.Layer):
    def __init__(self, 
        kernel_size=17,
        dilation_rate=1,
        use_bias=False,
        depthwise_initializer='glorot_uniform',
        name='', **kwargs):
        super().__init__(name=name,**kwargs)
        self.causal_pad = tf.keras.layers.ZeroPadding1D((dilation_rate*(kernel_size-1),0),name=name + '_pad')
        self.dw_conv = tf.keras.layers.DepthwiseConv1D(
                            kernel_size,
                            strides=1,
                            dilation_rate=dilation_rate,
                            padding='valid',
                            use_bias=use_bias,
                            depthwise_initializer=depthwise_initializer,
                            name=name + '_dwconv')
        self.supports_masking = True
        
    def call(self, inputs):
        x = self.causal_pad(inputs)
        x = self.dw_conv(x)
        return x


def Conv1DBlock(channel_size,
          kernel_size,
          dilation_rate=1,
          drop_rate=0.0,
          expand_ratio=2,
          se_ratio=0.25,
          activation='swish',
          name=None):
    '''
    efficient conv1d block, @hoyso48
    '''
    if name is None:
        name = str(tf.keras.backend.get_uid("mbblock"))
    # Expansion phase
    def apply(inputs):
        channels_in = tf.keras.backend.int_shape(inputs)[-1]
        channels_expand = channels_in * expand_ratio

        skip = inputs

        x = tf.keras.layers.Dense(
            channels_expand,
            use_bias=True,
            activation=activation,
            name=name + '_expand_conv')(inputs)

        # Depthwise Convolution
        x = CausalDWConv1D(kernel_size,
            dilation_rate=dilation_rate,
            use_bias=False,
            name=name + '_dwconv')(x)

        x = tf.keras.layers.BatchNormalization(momentum=0.95, name=name + '_bn')(x)

        x  = ECA()(x)

        x = tf.keras.layers.Dense(
            channel_size,
            use_bias=True,
            name=name + '_project_conv')(x)

        if drop_rate > 0:
            x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1), name=name + '_drop')(x)

        if (channels_in == channel_size):
            x = tf.keras.layers.add([x, skip], name=name + '_add')
        return x

    return apply


class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, dim=256, num_heads=4, dropout=0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.scale = self.dim ** -0.5
        self.num_heads = num_heads
        self.qkv = tf.keras.layers.Dense(3 * dim, use_bias=False)
        self.drop1 = tf.keras.layers.Dropout(dropout)
        self.proj = tf.keras.layers.Dense(dim, use_bias=False)
        self.supports_masking = True

    def call(self, inputs, mask=None):
        qkv = self.qkv(inputs)
        qkv = tf.keras.layers.Permute((2, 1, 3))(tf.keras.layers.Reshape((-1, self.num_heads, self.dim * 3 // self.num_heads))(qkv))
        q, k, v = tf.split(qkv, [self.dim // self.num_heads] * 3, axis=-1)

        attn = tf.matmul(q, k, transpose_b=True) * self.scale

        if mask is not None:
            mask = mask[:, None, None, :]

        attn = tf.keras.layers.Softmax(axis=-1)(attn, mask=mask)
        attn = self.drop1(attn)

        x = attn @ v
        x = tf.keras.layers.Reshape((-1, self.dim))(tf.keras.layers.Permute((2, 1, 3))(x))
        x = self.proj(x)
        return x


def TransformerBlock(dim=256, num_heads=4, expand=4, attn_dropout=0.2, drop_rate=0.2, activation='swish'):
    def apply(inputs):
        x = inputs
        x = tf.keras.layers.BatchNormalization(momentum=0.95)(x)
        x = MultiHeadSelfAttention(dim=dim,num_heads=num_heads,dropout=attn_dropout)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([inputs, x])
        attn_out = x

        x = tf.keras.layers.BatchNormalization(momentum=0.95)(x)
        x = tf.keras.layers.Dense(dim*expand, use_bias=False, activation=activation)(x)
        x = tf.keras.layers.Dense(dim, use_bias=False)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([attn_out, x])
        return x
    return apply


def get_model(max_len=MAX_LEN, dropout_step=0, dim=192):
    inp = tf.keras.Input((max_len,CHANNELS))
    # Include Masking layer to match training setup
    x = tf.keras.layers.Masking(mask_value=PAD)(inp)
    ksize = 17
    x = tf.keras.layers.Dense(dim, use_bias=False,name='stem_conv')(x)
    x = tf.keras.layers.BatchNormalization(momentum=0.95,name='stem_bn')(x)

    x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = TransformerBlock(dim,expand=2)(x)

    x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = TransformerBlock(dim,expand=2)(x)

    x = tf.keras.layers.Dense(dim*2,activation=None,name='top_conv')(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = LateDropout(0.8, start_step=dropout_step)(x)
    x = tf.keras.layers.Dense(NUM_CLASSES,name='classifier')(x)
    return tf.keras.Model(inp, x)


print("Building hoyso48 model (EXACT implementation)...")
print(f"Input: {MAX_LEN} frames × {CHANNELS} features (236 landmarks × 3)")
model = get_model()

print(f"\n📊 Model has {len(model.layers)} layers")
print("Layer names:", [layer.name for layer in model.layers[:10]], "...")

print("\nLoading weights from /content/...")
# Use by_name=True to load weights by layer name (more flexible)
try:
    model.load_weights(WEIGHTS_FILE, by_name=True, skip_mismatch=False)
    print("✅ Weights loaded successfully!")
except Exception as e:
    print(f"⚠️ Error loading weights: {e}")
    print("\nTrying with skip_mismatch=True...")
    model.load_weights(WEIGHTS_FILE, by_name=True, skip_mismatch=True)
    print("✅ Weights loaded (some layers skipped)")

print("\n✅ Model ready!")
print(f"Input shape: {model.input_shape}")
print(f"Output shape: {model.output_shape}")
print(f"\n📊 Model Summary:")
model.summary()

# Verify parameter count
total_params = model.count_params()
expected_params = 1_836_569
if total_params == expected_params:
    print(f"\n✅ PERFECT MATCH: {total_params:,} parameters!")
else:
    diff = abs(total_params - expected_params)
    print(f"\n⚠️ Parameters: {total_params:,} (expected: {expected_params:,}, diff: {diff:,})")

In [ ]:
# Cell 5: MediaPipe Landmark Extraction (hoyso48's format)

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# hoyso48 uses only LIPS (not full face)
LIPS_IDXS = np.array([
    61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
    291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
])  # 40 landmarks

LEFT_HAND_IDXS = np.arange(0, 21)   # 21 landmarks
POSE_IDXS = np.arange(0, 33)        # 33 landmarks
RIGHT_HAND_IDXS = np.arange(0, 21)  # 21 landmarks

# Total: 40 + 21 + 33 + 21 = 115 landmarks
# Padded to 236 landmarks (hoyso48's format)

def extract_landmarks(frame, holistic):
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(image_rgb)
    
    # Initialize with zeros (236 landmarks × 3 = 708 features)
    landmarks = np.zeros((236, 3), dtype=np.float32)
    
    # Lips (40 landmarks) → indices 0-39
    if results.face_landmarks:
        face_lms = np.array([[lm.x, lm.y, lm.z] for lm in results.face_landmarks.landmark])
        landmarks[0:40] = face_lms[LIPS_IDXS]
    
    # Left hand (21 landmarks) → indices 40-60
    if results.left_hand_landmarks:
        left_hand = np.array([[lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark])
        landmarks[40:61] = left_hand
    
    # Pose (33 landmarks) → indices 61-93
    if results.pose_landmarks:
        pose_lms = np.array([[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark])
        landmarks[61:94] = pose_lms
    
    # Right hand (21 landmarks) → indices 94-114
    if results.right_hand_landmarks:
        right_hand = np.array([[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark])
        landmarks[94:115] = right_hand
    
    # Remaining indices 115-235 stay as zeros (padding)
    
    return landmarks, results

def normalize_landmarks(landmarks):
    mean = landmarks.mean(axis=0)
    landmarks_centered = landmarks - mean
    std = landmarks_centered.std()
    if std > 0:
        return landmarks_centered / std
    return landmarks_centered

print("✅ MediaPipe ready (hoyso48's 236-landmark format)!")

In [ ]:
# Cell 6: Real-Time Detector (7 Inference Tricks)

MAX_LEN = 384  # Match model's sequence length
WINDOW_SIZE = 60
CONFIDENCE_THRESHOLD = 0.3
SMOOTHING_WINDOW = 5
MIN_SIGN_FRAMES = 20

class RealTimeSignDetector:
    def __init__(self, model, index_to_sign, max_len=384, window_size=60):
        self.model = model
        self.index_to_sign = index_to_sign
        self.max_len = max_len
        self.window_size = window_size
        self.frame_buffer = deque(maxlen=window_size)
        self.prediction_buffer = deque(maxlen=SMOOTHING_WINDOW)
        self.is_signing = False
        self.current_sign_frames = []
        self.fps_buffer = deque(maxlen=30)
    
    def detect_sign_activity(self, landmarks):
        left_hand = landmarks[LEFT_HAND_IDXS]
        right_hand = landmarks[RIGHT_HAND_IDXS]
        return (left_hand.sum() != 0) or (right_hand.sum() != 0)
    
    def process_frame(self, landmarks):
        landmarks_norm = normalize_landmarks(landmarks)
        self.frame_buffer.append(landmarks_norm)
        is_active = self.detect_sign_activity(landmarks_norm)
        
        if is_active:
            if not self.is_signing:
                self.is_signing = True
                self.current_sign_frames = []
            self.current_sign_frames.append(landmarks_norm)
        else:
            if self.is_signing and len(self.current_sign_frames) >= MIN_SIGN_FRAMES:
                return self.predict_sign()
            self.is_signing = False
            self.current_sign_frames = []
        
        return None, 0.0, []
    
    def predict_sign(self):
        if len(self.frame_buffer) < MIN_SIGN_FRAMES:
            return None, 0.0, []
        
        sequence = np.array(list(self.frame_buffer))
        
        if len(sequence) < self.max_len:
            padding = np.zeros((self.max_len - len(sequence), 543, 3))
            sequence = np.concatenate([sequence, padding], axis=0)
        else:
            sequence = sequence[-self.max_len:]
        
        # Flatten to (max_len, 543*3)
        sequence = sequence.reshape(self.max_len, -1)
        sequence = np.expand_dims(sequence, axis=0)
        predictions = self.model.predict(sequence, verbose=0)[0]
        
        top_k_indices = np.argsort(predictions)[-5:][::-1]
        top_k_probs = predictions[top_k_indices]
        top_k_signs = [(self.index_to_sign[str(idx)], prob) for idx, prob in zip(top_k_indices, top_k_probs)]
        
        self.prediction_buffer.append((top_k_indices[0], top_k_probs[0]))
        
        if len(self.prediction_buffer) >= SMOOTHING_WINDOW:
            avg_pred_idx = max(set([p[0] for p in self.prediction_buffer]), 
                             key=lambda x: sum([p[1] for p in self.prediction_buffer if p[0] == x]))
            avg_confidence = np.mean([p[1] for p in self.prediction_buffer if p[0] == avg_pred_idx])
            
            if avg_confidence >= CONFIDENCE_THRESHOLD:
                predicted_sign = self.index_to_sign[str(avg_pred_idx)]
                return predicted_sign, avg_confidence, top_k_signs
        
        return None, 0.0, top_k_signs

# Create reverse mapping
index_to_sign = {str(v): k for k, v in sign_to_index.items()}

print("✅ Detector ready!")

In [ ]:
# Cell 7: Webcam Inference (Run this to start!)

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Initialize
detector = RealTimeSignDetector(model, index_to_sign, max_len=MAX_LEN, window_size=WINDOW_SIZE)
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

print("🎥 Webcam Inference Ready!")
print("Click 'Capture' button to test a frame")
print("-" * 50)

# Capture and process
try:
    filename = take_photo()
    frame = cv2.imread(filename)
    
    landmarks, results = extract_landmarks(frame, holistic)
    predicted_sign, confidence, top_k = detector.process_frame(landmarks)
    
    if predicted_sign:
        print(f"✅ Detected: {predicted_sign} ({confidence:.2%})")
        print(f"Top-5: {', '.join([f'{s}({p:.2%})' for s, p in top_k[:5]])}")
    else:
        print("No sign detected (try performing a sign)")
        
except Exception as e:
    print(f"Error: {e}")
finally:
    holistic.close()

print("\n💡 Note: For continuous inference, use a local setup with OpenCV.")

## Notes

**Colab Limitations:**

- Webcam access in Colab is limited (single frame capture)
- For real-time continuous inference, run locally with `cv2.VideoCapture(0)`

**Next Steps:**
1. Wait for your fine-tuned 200-class model from Kaggle
2. Replace the model file and labels
3. Test with your custom vocabulary

**Performance:**
- GPU inference: ~30-50ms per prediction
- Expected FPS: 20-30 FPS on local setup